In [1]:
!pip install matplotlib
!pip install statsmodels

In [2]:
from pathlib import Path

from dateutil.parser import parse 
from matplotlib.pyplot import cm
from plotly.subplots import make_subplots
from scipy import signal
from pandas.api.indexers import BaseIndexer
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.api import OLS
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

import plotly.express as px
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import os
import warnings 
import datetime


import warnings
warnings.filterwarnings('ignore')

In [3]:
csvs = Path("/home/dmacs/Documents/Pranav_Phanindra/Data_Scraping_Final/Data_Part_2/Collated/CSVs")

#Directory containing all the files
DataFiles = os.listdir(csvs)

CropData = pd.read_csv(csvs / '12_Mustard.csv' , parse_dates = ['Price Date'])
CropData = CropData.iloc[:,1:]

CropData['Price Date'] =  pd.to_datetime(CropData['Price Date'])

CropData['Yr-mon'] = CropData['Price Date'].dt.strftime('%Y%m')
CropData['Yr-mon'] = pd.to_datetime(CropData['Yr-mon'], format = '%Y%m')

CropData['Year'] = CropData.set_index('Yr-mon').index.year
CropData['Month'] = CropData.set_index('Yr-mon').index.month

CropData.set_index('Price Date')

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),State,Yr-mon,Year,Month
Price Date,,,,,,,,,,,,
2009-02-20,Kurnool,Adoni,Mustard,Other,FAQ,1550.0,1600.0,1575.0,Andhra Pradesh,2009-02-01,2009,2
2013-05-03,Kurnool,Alur,Mustard,Other,Medium,2918.0,2918.0,2918.0,Andhra Pradesh,2013-05-01,2013,5
2013-03-25,Kurnool,Alur,Mustard,Mustard,Medium,3018.0,3018.0,3018.0,Andhra Pradesh,2013-03-01,2013,3
2013-03-25,Kurnool,Alur,Mustard,Other,Medium,3018.0,3018.0,3018.0,Andhra Pradesh,2013-03-01,2013,3
2012-04-22,Kurnool,Alur,Mustard,Mustard,Medium,2598.0,2598.0,2598.0,Andhra Pradesh,2012-04-01,2012,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-05,Coochbehar,Toofanganj,Mustard,Mustard,FAQ,4300.0,4500.0,4400.0,West Bengal,2016-06-01,2016,6
2016-06-04,Coochbehar,Toofanganj,Mustard,Mustard,FAQ,4300.0,4500.0,4400.0,West Bengal,2016-06-01,2016,6
2016-06-03,Coochbehar,Toofanganj,Mustard,Mustard,FAQ,4300.0,4500.0,4400.0,West Bengal,2016-06-01,2016,6


In [6]:
CropData_Summarized = CropData.groupby(['Yr-mon'],as_index = False).mean()

CropData_Summarized['Year'] = CropData_Summarized.set_index('Yr-mon').index.year
CropData_Summarized['Month'] = CropData_Summarized.set_index('Yr-mon').index.month
"""
import datetime
mydate = datetime.datetime.now()
mydate.strftime("%B")

CropData_Summarized['Month_name'] =  CropData_Summarized.set_index('Yr-mon').index.strftime('%B')
"""


years = CropData_Summarized['Year'].unique()


month = CropData_Summarized['Month'].unique()

print(years)
print(month)


[2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022]
[ 4  5  6  7  8  9 10 11 12  1  2  3]


In [7]:
CropData_Summarized

,Yr-mon,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month
0,2008-04-01,1972.009636,2205.367174,2245.057074,2008,4
1,2008-05-01,2064.281171,2281.468051,2334.040883,2008,5
2,2008-06-01,2275.346497,2465.060944,2572.880386,2008,6
3,2008-07-01,2256.718133,2478.504936,2598.776824,2008,7
4,2008-08-01,2326.431642,2486.026648,2595.234415,2008,8
...,...,...,...,...,...,...
163,2021-11-01,7147.131780,7425.545886,7324.340586,2021,11
164,2021-12-01,6895.516610,7190.095399,7096.493960,2021,12
165,2022-01-01,6612.206225,6959.096469,6847.179651,2022,1
166,2022-02-01,6275.087460,6867.760500,6640.370247,2022,2


In [8]:
CropData_Summarized['SMA(12)'] = CropData_Summarized.rolling(12,center = True).mean()['Modal Price (Rs./Quintal)']
CropData_Summarized['Price-SMA(12)'] = CropData_Summarized['Modal Price (Rs./Quintal)']-CropData_Summarized[CropData_Summarized['SMA(12)'].notnull()]['SMA(12)']



In [10]:
CropData_Summarized[CropData_Summarized['SMA(12)'].notnull()]

,Yr-mon,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month,SMA(12),Price-SMA(12)
6,2008-10-01,2199.782344,2370.968582,2564.292059,2008,10,2426.091588,138.200471
7,2008-11-01,2268.931493,2411.326838,2560.765303,2008,11,2418.680233,142.085071
8,2008-12-01,2240.490319,2419.771060,2576.867611,2008,12,2412.419330,164.448282
9,2009-01-01,1983.314762,2208.951741,2348.054467,2009,1,2387.354526,-39.300059
10,2009-02-01,1830.037688,2039.327834,2147.439906,2009,2,2360.518100,-213.078194
...,...,...,...,...,...,...,...,...
158,2021-06-01,5989.137463,6300.593042,6177.388681,2021,6,6111.001863,66.386818
159,2021-07-01,6166.852757,6452.764186,6344.841906,2021,7,6279.931161,64.910745
160,2021-08-01,6545.549413,6840.657049,6732.234623,2021,8,6417.337933,314.896690
161,2021-09-01,7039.999173,7348.164655,7235.832369,2021,9,6541.561708,694.270661


In [12]:
PrSmaMon = pd.DataFrame()
PrSmaMon['Seasonal_Average'] = CropData_Summarized[CropData_Summarized['SMA(12)'].notnull()].groupby('Month').mean()['Price-SMA(12)']
k = PrSmaMon.mean()

PrSmaMon = PrSmaMon.subtract(k) 


print(PrSmaMon.mean())
print(PrSmaMon)

Seasonal_Average    2.368476e-15
dtype: float64
       Seasonal_Average
Month                  
1             64.202891
2           -115.131920
3           -236.586464
4           -161.948485
5            -77.302355
6            -59.444214
7              3.559216
8             84.079635
9            108.665077
10           120.855210
11           133.058174
12           135.993234


In [18]:
"{:.50f}".format(float(PrSmaMon.mean()))

'0.00000000000000236847578586700048809359668842511576'

In [19]:
outer_join_df=pd.merge(CropData_Summarized, PrSmaMon.reset_index() , on='Month', how='outer')
outer_join_df

,Yr-mon,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month,SMA(12),Price-SMA(12),Seasonal_Average
0,2008-04-01,1972.009636,2205.367174,2245.057074,2008,4,NaN,NaN,-161.948485
1,2009-04-01,1834.209134,1974.845651,2156.120807,2009,4,2314.331982,-158.211175,-161.948485
2,2010-04-01,1708.761387,1830.935290,2084.289436,2010,4,2255.400420,-171.110984,-161.948485
3,2011-04-01,1813.669792,1950.141547,2242.217045,2011,4,2388.683903,-146.466858,-161.948485
4,2012-04-01,2855.985227,3077.535771,3317.324299,2012,4,3211.518395,105.805903,-161.948485
...,...,...,...,...,...,...,...,...,...
163,2018-03-01,3372.185184,3618.523506,3521.648014,2018,3,3575.116631,-53.468617,-236.586464
164,2019-03-01,3319.762289,3599.290994,3486.515963,2019,3,3689.722800,-203.206837,-236.586464
165,2020-03-01,3574.871048,3856.149467,3740.869816,2020,3,4008.327970,-267.458154,-236.586464
166,2021-03-01,4787.342463,5131.809942,4989.495575,2021,3,5511.642811,-522.147236,-236.586464


In [29]:
outer_join_df.sort_values('Yr-mon' , inplace= True)
outer_join_df.set_index('Yr-mon',inplace= True)

In [31]:
outer_join_df[outer_join_df['SMA(12)'].notnull()]

,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month,SMA(12),Price-SMA(12),Seasonal_Average
Yr-mon,,,,,,,,
2008-10-01,2199.782344,2370.968582,2564.292059,2008,10,2426.091588,138.200471,120.855210
2008-11-01,2268.931493,2411.326838,2560.765303,2008,11,2418.680233,142.085071,133.058174
2008-12-01,2240.490319,2419.771060,2576.867611,2008,12,2412.419330,164.448282,135.993234
2009-01-01,1983.314762,2208.951741,2348.054467,2009,1,2387.354526,-39.300059,64.202891
2009-02-01,1830.037688,2039.327834,2147.439906,2009,2,2360.518100,-213.078194,-115.131920
...,...,...,...,...,...,...,...,...
2021-06-01,5989.137463,6300.593042,6177.388681,2021,6,6111.001863,66.386818,-59.444214
2021-07-01,6166.852757,6452.764186,6344.841906,2021,7,6279.931161,64.910745,3.559216
2021-08-01,6545.549413,6840.657049,6732.234623,2021,8,6417.337933,314.896690,84.079635


In [34]:
outer_join_df['Deseasonlaised_Data'] = outer_join_df[outer_join_df['SMA(12)'].notnull()]['Modal Price (Rs./Quintal)'] - outer_join_df[outer_join_df['SMA(12)'].notnull()]['Seasonal_Average']

In [40]:
outer_join_df[outer_join_df['SMA(12)'].notnull()]

,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month,SMA(12),Price-SMA(12),Seasonal_Average,Deseasonlaised_Data
Yr-mon,,,,,,,,,
2008-10-01,2199.782344,2370.968582,2564.292059,2008,10,2426.091588,138.200471,120.855210,2443.436849
2008-11-01,2268.931493,2411.326838,2560.765303,2008,11,2418.680233,142.085071,133.058174,2427.707129
2008-12-01,2240.490319,2419.771060,2576.867611,2008,12,2412.419330,164.448282,135.993234,2440.874377
2009-01-01,1983.314762,2208.951741,2348.054467,2009,1,2387.354526,-39.300059,64.202891,2283.851576
2009-02-01,1830.037688,2039.327834,2147.439906,2009,2,2360.518100,-213.078194,-115.131920,2262.571826
...,...,...,...,...,...,...,...,...,...
2021-06-01,5989.137463,6300.593042,6177.388681,2021,6,6111.001863,66.386818,-59.444214,6236.832895
2021-07-01,6166.852757,6452.764186,6344.841906,2021,7,6279.931161,64.910745,3.559216,6341.282690
2021-08-01,6545.549413,6840.657049,6732.234623,2021,8,6417.337933,314.896690,84.079635,6648.154988


In [44]:
fig = go.Figure()
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=outer_join_df[outer_join_df['SMA(12)'].notnull()].index,
                         y=outer_join_df[outer_join_df['SMA(12)'].notnull()]['Deseasonlaised_Data'],
                         mode='lines',name = "Deseasonlaised_Data") , row=1, col=1)
    
    

fig.add_trace(go.Scatter(x=outer_join_df[outer_join_df['SMA(12)'].notnull()].index,
                         y=outer_join_df[outer_join_df['SMA(12)'].notnull()]['Modal Price (Rs./Quintal)'],
                         mode='lines' , name = "Modal Price"),row=1, col=1)

In [45]:
outer_join_df['Stationary'] = outer_join_df[outer_join_df['SMA(12)'].notnull()]['Deseasonlaised_Data'] - outer_join_df[outer_join_df['SMA(12)'].notnull()]['SMA(12)']

In [48]:
outer_join_df[outer_join_df['SMA(12)'].notnull()]

,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Year,Month,SMA(12),Price-SMA(12),Seasonal_Average,Deseasonlaised_Data,Stationary
Yr-mon,,,,,,,,,,
2008-10-01,2199.782344,2370.968582,2564.292059,2008,10,2426.091588,138.200471,120.855210,2443.436849,17.345261
2008-11-01,2268.931493,2411.326838,2560.765303,2008,11,2418.680233,142.085071,133.058174,2427.707129,9.026896
2008-12-01,2240.490319,2419.771060,2576.867611,2008,12,2412.419330,164.448282,135.993234,2440.874377,28.455047
2009-01-01,1983.314762,2208.951741,2348.054467,2009,1,2387.354526,-39.300059,64.202891,2283.851576,-103.502950
2009-02-01,1830.037688,2039.327834,2147.439906,2009,2,2360.518100,-213.078194,-115.131920,2262.571826,-97.946274
...,...,...,...,...,...,...,...,...,...,...
2021-06-01,5989.137463,6300.593042,6177.388681,2021,6,6111.001863,66.386818,-59.444214,6236.832895,125.831032
2021-07-01,6166.852757,6452.764186,6344.841906,2021,7,6279.931161,64.910745,3.559216,6341.282690,61.351529
2021-08-01,6545.549413,6840.657049,6732.234623,2021,8,6417.337933,314.896690,84.079635,6648.154988,230.817055


In [50]:
fig = go.Figure()
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=outer_join_df[outer_join_df['SMA(12)'].notnull()].index,
                         y=outer_join_df[outer_join_df['SMA(12)'].notnull()]['Stationary'],
                         mode='lines',name = "Stationary") , row=1, col=1)
    
fig.add_trace(go.Scatter(x=outer_join_df[outer_join_df['SMA(12)'].notnull()].index,
                         y=outer_join_df[outer_join_df['SMA(12)'].notnull()]['Deseasonlaised_Data'],
                         mode='lines',name = "Deseasonlaised_Data") , row=1, col=1)
    

fig.add_trace(go.Scatter(x=outer_join_df[outer_join_df['SMA(12)'].notnull()].index,
                         y=outer_join_df[outer_join_df['SMA(12)'].notnull()]['Modal Price (Rs./Quintal)'],
                         mode='lines' , name = "Modal Price"),row=1, col=1)